In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import os
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import explained_variance_score
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

In [19]:
path = "\\".join(os.getcwd().split("\\")[:-1])
dataset = pd.read_csv(path + "\\finalData.csv")
dataset = dataset[dataset['civic_issue']==1]
dataset = dataset[['description','category']]
dataset.drop_duplicates(subset='description',inplace=True,keep=False)
dataset.count()

description    15234
category       15234
dtype: int64

 ## Preprocessing the Description 
 
 The preprocessing is done in 4 steps:

    - removing punctuation
    - removing stopwords like 'the', 'this','as',etc
    - conversion of the entire text to lower case
    - Stemming: reducing the number of inflectional forms of words by reducing all to their common stem.For example, 'argue','arguing','argued' are all reduced to 'argu'
    - Splitting dataset into train and cross validation sets

In [20]:
def preprocess():
    stemmer = PorterStemmer()
    words = stopwords.words("english")
    dataset['processedtext'] = dataset['description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

preprocess()
data = dataset[['processedtext','category']]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data['processedtext'],data['category'], test_size=0.20, random_state=42)

### Defining functions to calculate model metrics

In [22]:
def auc_roc(classifier):
    pred_probs = classifier.predict_proba(test_tfidf)
    train_probs = classifier.predict_proba(train_tfidf)[:,1]
    noSkillProb = [0 for _ in range(len(y_test))]
    lr_probs = pred_probs[:,1]

    noSkillAUC = roc_auc_score(y_test,noSkillProb)
    logRegAUC = roc_auc_score(y_test,lr_probs)

    print('No Skill: ROC AUC=%.3f' % (noSkillAUC))
    print('Logistic: ROC AUC=%.3f' % (logRegAUC))

    #FPR,TPR,thresholds = roc_curve(y_test,)
    ns_fpr, ns_tpr, _ = roc_curve(y_test, noSkillProb)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    t_fpr,t_tpr,_ = roc_curve(y_train,train_probs)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
    plt.plot(t_fpr,t_tpr,marker='*',label='Training')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [23]:

def model_metrics(classifier,y_test,pred,score):
    print("Accuracy:", score*100, "%")
    # print("Precision:",precision_score(y_test,pred)*100,"%")
    # print("Recall:",recall_score(y_test,pred)*100,"%")
    # print("F1 Score:",f1_score(y_test,pred)*100,"%")
    # print("MSE:",mean_squared_error(y_test,pred)*100,"%")
    # print("Explained Variance Regression Score:", explained_variance_score(y_test,pred))
    # auc_roc(classifier)

## Training the Linear SVC Model and Cross Validation

In [25]:
def train_SVC(train_tfIdf, y_train):
    #building text classification model using Linear Kernel SVC Classifier (has highest accuracy)
    classifier = SVC(kernel='linear') #accuracy obtained for linear kernel = 83.28%
    classifier.fit(train_tfIdf, y_train) #fitting the classifier onto the training data
    filename = "linearkernelSVC.sav"
    pickle.dump(classifier,open(filename,"wb"))

def predict_cat():  
    # X_train: description data for training
    # y_train: corresponding categories for training
    # X_test and y_test: description and category for testing
    
    # Vectorizing the train and test data using TfIDf vectorization
    # TfIdf - Text Frequency Inverse Document Freqeuncy : vectorizes based on frequency across the current text document but less frequency across multiple documents

    vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7) #initializing the vector
    train_tfIdf = vectorizer_tfidf.fit_transform(X_train.values.astype('U')) #astype('U') converts the dataframe into a Unicode array
    test_tfIdf = vectorizer_tfidf.transform(X_test.values.astype('U'))#transforming the text into frequency vectors
    
    # train_SVC(train_tfIdf, y_train)

    classifier = pickle.load(open("linearkernelSVC.sav","rb"))
    predictions = classifier.predict(test_tfIdf) #predictions made on the unseen data
    train_score = classifier.score(train_tfIdf, y_train)
    print("\n\nTrain Accuracy:",train_score*100,"%\n\n")
    score = classifier.score(test_tfIdf,y_test)
    model_metrics(classifier,y_test,predictions,score)

predict_cat()



Train Accuracy: 92.64790350373349 %


Accuracy: 85.16573679028554 %


In [19]:
print(dataset['category'].unique())

['Waste/Garbage' 'Air' 'Streetlights' 'Traffic/Parking' 'Sewage'
 'Potholes' 'Electricity' 'Water' 'Plastic' 'Sanitation']


In [24]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 500
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dataset['description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 20201 unique tokens.


In [30]:
X = tokenizer.texts_to_sequences(dataset['description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (15234, 500)


In [31]:
Y = pd.get_dummies(dataset['category']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (15234, 10)


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(12187, 500) (12187, 10)
(3047, 500) (3047, 10)


In [45]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
172/172 [==============================] - 234s 1s/step - loss: 1.3335 - accuracy: 0.6096 - val_loss: 0.9802 - val_accuracy: 0.6891
Epoch 2/5
172/172 [==============================] - 167s 971ms/step - loss: 0.7730 - accuracy: 0.7567 - val_loss: 0.7404 - val_accuracy: 0.7834
Epoch 3/5
172/172 [==============================] - 164s 956ms/step - loss: 0.4985 - accuracy: 0.8476 - val_loss: 0.6695 - val_accuracy: 0.7900
Epoch 4/5
172/172 [==============================] - 168s 975ms/step - loss: 0.3694 - accuracy: 0.8901 - val_loss: 0.6348 - val_accuracy: 0.8236
Epoch 5/5
172/172 [==============================] - 168s 975ms/step - loss: 0.2710 - accuracy: 0.9233 - val_loss: 0.6729 - val_accuracy: 0.8236


In [46]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

96/96 [==============================] - 11s 111ms/step - loss: 0.6886 - accuracy: 0.8192
Test set
  Loss: 0.689
  Accuracy: 0.819


In [52]:
new_complaint = ['The Bangalore traffic is adding to the pollution in the city.'] #bruh it isnt really accurate
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = dataset['category']
print(pred, labels[np.argmax(pred)])

[[0.06495962 0.01019261 0.00348917 0.01619492 0.01081865 0.00263569
  0.04165866 0.81442034 0.03258398 0.0030464 ]] Traffic/Parking
